In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = r"C:\Users\Poovesh Kumar M\Downloads\Planning\1_Data\2_Sep_02_09_2024\majestic pure_Amazon_com_weekly_data.csv"

# Read the CSV file into a DataFrame
df_base = pd.read_csv(file_path)

df_base = df_base[df_base['Week_Start_date'] <= '2024-08-26']

# Display the DataFrame
df_base.head()

,velocity_bucket,asin,Week_Start_date,Week_End_date,organic_gmv_units,inorganic_gmv_units,asp_week,ad_spend,inventory_Availability%,gmv,inorganic_gmv,organic_gmv,gmv_units,inventory_available
0,1_SF,B00PMR3QF2,2022-09-26,2022-10-02,220.8734,145.9917,18.886359,1626.1800,0.0,6950.18,2757.6640,4171.0360,368,0
1,1_SF,B00PMR3QF2,2022-10-03,2022-10-09,756.0293,464.0168,18.895069,4042.8753,0.0,23184.25,8768.5915,14284.1985,1227,0
2,1_SF,B00PMR3QF2,2022-10-10,2022-10-16,834.9785,813.4476,16.091245,6729.7078,0.0,26631.01,13023.9672,13496.8928,1655,0
3,1_SF,B00PMR3QF2,2022-10-17,2022-10-23,883.1982,597.8767,16.000302,4171.1138,0.0,23824.45,9565.7531,14131.8569,1489,0
4,1_SF,B00PMR3QF2,2022-10-24,2022-10-30,1034.3934,694.0630,16.050614,4477.2391,0.0,27960.17,11141.9337,16600.6563,1742,0


In [3]:
# print(len(df_base))
print("Asin count in base data: ", df_base.asin.nunique())
df_base_with_gmv_gt_zero = df_base[df_base['gmv_units']>0]
print("Asin count where gmv_units > 0: ", df_base_with_gmv_gt_zero.asin.nunique())

Asin count in base data:  450
Asin count where gmv_units > 0:  270


In [4]:
sales_week_df = df_base_with_gmv_gt_zero.groupby('asin').agg({ 'Week_Start_date':['nunique','min', 'max'],'gmv_units':['mean',np.std]}).reset_index()
sales_week_df.columns = [' '.join(col).strip() for col in sales_week_df.columns.values]
sales_week_df

,asin,Week_Start_date nunique,Week_Start_date min,Week_Start_date max,gmv_units mean,gmv_units std
0,B00PMR3QF2,101,2022-09-26,2024-08-26,2323.316832,948.207846
1,B00PV15BPW,101,2022-09-26,2024-08-26,1707.940594,867.192722
2,B00Q96XGUU,94,2022-09-26,2024-08-05,19.840426,24.107422
3,B00QR6SS6O,101,2022-09-26,2024-08-26,821.425743,375.722354
4,B00QR7FTLU,101,2022-09-26,2024-08-26,92.188119,65.292528
...,...,...,...,...,...,...
265,B0D7HX5FJF,8,2024-07-08,2024-08-26,244.125000,182.817190
266,B0D7HZ6883,8,2024-07-08,2024-08-26,238.125000,269.090764
267,B0D7HZR4XL,5,2024-07-08,2024-08-26,17.600000,10.620734
268,B0D811N8HV,7,2024-07-15,2024-08-26,78.142857,29.779347


In [5]:
sales_week_df['Week_Start_date max'] = pd.to_datetime(sales_week_df['Week_Start_date max'], format='%Y-%m-%d')
sales_week_df['Week_Start_date min'] = pd.to_datetime(sales_week_df['Week_Start_date min'], format='%Y-%m-%d')
sales_week_df['n_weeks'] = ((sales_week_df['Week_Start_date max'] - sales_week_df['Week_Start_date min']).dt.days)/7 
# sales_week_df
sales_week_df['ADI'] =  sales_week_df['n_weeks'] / (sales_week_df['Week_Start_date nunique'])

# eg: B0D7HZR4XL - Some edge case here ADI going > 1

sales_week_df['CV_2'] = (sales_week_df['gmv_units std']/sales_week_df['gmv_units mean'])**2

sales_week_df


,asin,Week_Start_date nunique,Week_Start_date min,Week_Start_date max,gmv_units mean,gmv_units std,n_weeks,ADI,CV_2
0,B00PMR3QF2,101,2022-09-26,2024-08-26,2323.316832,948.207846,100.0,0.990099,0.166567
1,B00PV15BPW,101,2022-09-26,2024-08-26,1707.940594,867.192722,100.0,0.990099,0.257802
2,B00Q96XGUU,94,2022-09-26,2024-08-05,19.840426,24.107422,97.0,1.031915,1.476385
3,B00QR6SS6O,101,2022-09-26,2024-08-26,821.425743,375.722354,100.0,0.990099,0.209217
4,B00QR7FTLU,101,2022-09-26,2024-08-26,92.188119,65.292528,100.0,0.990099,0.501623
...,...,...,...,...,...,...,...,...,...
265,B0D7HX5FJF,8,2024-07-08,2024-08-26,244.125000,182.817190,7.0,0.875000,0.560802
266,B0D7HZ6883,8,2024-07-08,2024-08-26,238.125000,269.090764,7.0,0.875000,1.276990
267,B0D7HZR4XL,5,2024-07-08,2024-08-26,17.600000,10.620734,7.0,1.400000,0.364153
268,B0D811N8HV,7,2024-07-15,2024-08-26,78.142857,29.779347,6.0,0.857143,0.145228


In [6]:
def classification(x):
    if x['ADI']<1.32 and x['CV_2']<0.49:
        return 'Smooth'
    elif x['ADI']>=1.32 and x['CV_2']<0.49:
        return 'Intermittent'
    elif x['ADI']<1.32 and x['CV_2']>=0.49:
        return 'Erratic'
    elif x['ADI']>=1.32 and x['CV_2']>=0.49:
        return 'Lumpy'
    else:
        return 'NA'

In [7]:
sales_week_df['classification'] = sales_week_df.apply(lambda x: classification(x), axis=1)
sales_week_df

,asin,Week_Start_date nunique,Week_Start_date min,Week_Start_date max,gmv_units mean,gmv_units std,n_weeks,ADI,CV_2,classification
0,B00PMR3QF2,101,2022-09-26,2024-08-26,2323.316832,948.207846,100.0,0.990099,0.166567,Smooth
1,B00PV15BPW,101,2022-09-26,2024-08-26,1707.940594,867.192722,100.0,0.990099,0.257802,Smooth
2,B00Q96XGUU,94,2022-09-26,2024-08-05,19.840426,24.107422,97.0,1.031915,1.476385,Erratic
3,B00QR6SS6O,101,2022-09-26,2024-08-26,821.425743,375.722354,100.0,0.990099,0.209217,Smooth
4,B00QR7FTLU,101,2022-09-26,2024-08-26,92.188119,65.292528,100.0,0.990099,0.501623,Erratic
...,...,...,...,...,...,...,...,...,...,...
265,B0D7HX5FJF,8,2024-07-08,2024-08-26,244.125000,182.817190,7.0,0.875000,0.560802,Erratic
266,B0D7HZ6883,8,2024-07-08,2024-08-26,238.125000,269.090764,7.0,0.875000,1.276990,Erratic
267,B0D7HZR4XL,5,2024-07-08,2024-08-26,17.600000,10.620734,7.0,1.400000,0.364153,Intermittent
268,B0D811N8HV,7,2024-07-15,2024-08-26,78.142857,29.779347,6.0,0.857143,0.145228,Smooth


In [8]:
category_analysis_df = sales_week_df.groupby('classification').agg({ 'asin':'nunique'}).reset_index()
category_analysis_df

,classification,asin
0,Erratic,97
1,Intermittent,18
2,Lumpy,12
3,NA,11
4,Smooth,132


In [9]:
sales_week_df.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/ASIN_classification_weekly.csv', index=False)
